In [170]:
import pandas as pd
from datetime import datetime
from scipy.optimize import fsolve

# Define constants and file paths
path212 = r"C:\Users\nikit\Downloads\212.csv"
ticker212 = {
    "Xtrackers MSCI World Information Technology (Acc)" : "XDWT",
    "iShares Edge MSCI World Quality Factor (Acc)" : "IWFQ",
    "Lyxor DJ Global Titans 50" : "MGT",
    "iShares Core S&P 500 (Acc)" : "SXR8",
    "Invesco EQQQ Nasdaq-100 (Dist)" : "EQQQ",
    "iShares USD Short Duration High Yield Corp Bond (Dist)" : "SDHY",
    "Xtrackers MSCI Japan (Acc)" : "XMK9",
    "iShares Core EURO STOXX 50 DE (Dist)" : "EXW1",
    "Amundi Msci Emerging Markets (Acc)" : "AUEG",
}

# Function to load and preprocess data
def load_and_preprocess_data(path, ticker_dict):
    df = pd.read_csv(path)
    df = df[df['Ticker'].isin(ticker_dict.values())]
    df['Time'] = pd.to_datetime(df['Time'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f").strftime("%Y-%m-%d")))
    return df

# Function to calculate invested amount
def calculate_invested(df):
    df_copy = df.copy()
    df_copy = df_copy.drop(columns=['Ticker', "No. of shares", 'Price / share', 'Exchange rate'])
    gdf = df_copy.groupby(['Time']).sum().reset_index()
    return gdf, gdf['Total'].sum()

# Function to calculate current balance
def calculate_balance(df, recent_date):
    df_copy = df.copy()
    df_copy = df_copy.drop(columns=['Time', 'Total'])
    df_grouped = df_copy.groupby(['Ticker']).sum().reset_index()
    today_prices = df[df['Time'] == recent_date].copy()
    today_prices['price_eur'] = today_prices['Price / share'] / today_prices['Exchange rate']
    today_prices = today_prices[['Ticker', 'price_eur']]
    merged_df = pd.merge(df_grouped, today_prices, on='Ticker')
    merged_df['Capitalisation'] = merged_df['No. of shares'] * merged_df['price_eur']
    return merged_df['Capitalisation'].sum()

# Function to solve for the annual rate
def solve_annual_rate(df, balance, current_date):
    df['Days'] = (current_date - df['Time']).dt.days
    df['Years'] = df['Days'] / 365

    def equation(r):
        return sum(df['Total'] * (1 + r) ** df['Years']) - balance

    initial_guess = 0.1
    annual_rate_solution = fsolve(equation, initial_guess)[0]
    return annual_rate_solution

# Main function to orchestrate the workflow
def main():
    df = load_and_preprocess_data(path212, ticker212)
    recent_date = df['Time'].max()
    gdfs, invested = calculate_invested(df)
    balance = calculate_balance(df, recent_date)
    annual_rate = solve_annual_rate(gdfs, balance, recent_date)
    
    # Save the preprocessed data
    gdfs.to_csv(r"C:\Users\nikit\Downloads\lol.csv", index=False)
    
    total_period = gdfs['Years'].max() - gdfs['Years'].min()
    print(f"For the period of {total_period:.2f} years")
    print(f"\t Balance/Invested {100*(balance/invested - 1):.2f}%")
    print(f"\t Averaged Annual Rate {total_period * 100*annual_rate:.2f}%")
    print("Annually:")
    print(f"\t Balance/Invested {100*(balance/invested - 1)/total_period:.2f}%")
    print(f"\t Averaged Annual Rate: {100*annual_rate:.2f}%")

if __name__ == "__main__":
    main()


For the period of 0.33 years
	 Balance/Invested 5.46%
	 Averaged Annual Rate 11.16%
Annually:
	 Balance/Invested 16.46%
	 Averaged Annual Rate: 33.68%


In [ ]:
# 12/06/2024
# For the period of 0.33 years
# 	 Balance/Invested 5.46%
# 	 Averaged Annual Rate 11.16%
# Annually:
# 	 Balance/Invested 16.46%
# 	 Averaged Annual Rate: 33.68%